In [1]:
%pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################

resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 Para saber a data da próxima Imersão IA com Google Gemini da Alura, a melhor forma é verificar diretamente no site da Alura:

*   **Site da Alura:** Procure na seção de cursos, imersões ou eventos. Geralmente, eles têm um calendário ou uma página específica para anunciar as próximas datas.
*   **Redes Sociais da Alura:** Siga a Alura no LinkedIn, Instagram, Twitter e outras redes. Eles costumam anunciar as novidades por lá.
*   **Comunidade Alura:** Se você já é aluno, participe da comunidade da Alura, pois as datas dos eventos são frequentemente divulgadas por lá.

Assim, você terá a informação mais precisa e atualizada.

In [6]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Response:\n {response.text}"))

Response:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. Durante este período, você terá acesso a 5 aulas online com o objetivo de aprender na prática a criar prompts poderosos, automatizar tarefas e desenvolver um sistema de agentes autônomos que trabalham sozinhos, executando tarefas por você.

As inscrições estão abertas até o dia 11 de maio e podem ser feitas no [site da Alura](https://www.alura.com.br/imersao-ia-google-gemini-ii). O curso é totalmente online e gratuito, e não há pré-requisitos para participar. Ao final, você receberá um certificado Alura + Google.


In [7]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura', 'Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: starten.tech, alura.com.br, tecmundo.com.br, educacao.mg.gov.br



In [8]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google_adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [9]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [10]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [11]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [44]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def ctb_1_prompt(topico, data_de_hoje):
    melhorador_de_prompt = Agent(
        name="ctb_1_prompt",
        model="gemini-2.0-flash",
        description = "Recebe o prompt do aluno; identifica o tipo; analisa o prompt e identifica o principal objetivo; reformula o prompt",
        instruction = """
          Tarefas:
            tarefa_1:
              Ao receber o prompt enviado pelo aluno, defina o tipo de comando, se é: pergunta, comando ou instrução. Feito isso, guarde o prompt e a definição que você deu a ele com o seguinte nome: informação_1; Para usa-lo na tarefa_2.

            tarefa_2:
              Feita a "tarefa_1" analise o material "informação_1" q foi guardado na "tarefa_1"; Ponha-o em questionamento em relação aos seguintes tópicos:
                Tema Central: Qual é o assunto principal abordado.
                Formato de Resposta Esperado: Texto explicativo, passo a passo, lista, comparação, etc.
                Objetivo Didático: O que o aluno parece querer entender, resolver ou aprender com aquilo.

            tarefa_3:
              Crie um novo prompt baseado nas respostas do questionamento feito na "tarefa_2". (subistituindo o prompt mandado pelo aluno).
        """
    )
    entrada_do_ctb_melhorador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    lancamentos = call_agent(melhorador_de_prompt, entrada_do_ctb_melhorador)
    return lancamentos
    # Executa o agente

In [43]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def ctb_2_dados(topico, lancamentos_buscados):
    coletor_de_dados = Agent(
        name="ctb_2_dados",
        model="gemini-2.0-flash",
        description="Agente responsável pela coleta de dados referente a salicitação",
        tools=[google_search],
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
          Regras:
            1. Ao buscar no google, faça a busca e coleta de dados, tendo como primissa: Informações mais atuais; Sites que tenham boa avaliação; Sites oficiais a respeito do conteúdo em questão.
            2. Ao coletar os documentos, analiseos para manter a qualidade e confiança dos conteúdos.

          tarefa_1:
            Analise o prompt gerado pelo "ctb_1_prompt", separe em tema e subtemas o contúdo solicitado no prompt do "ctb_1_prompt".
            Após analisar e separar o tema com os subtemas, faça uma busca no google para coletar dados (textos, documentos e links) diretamente ligados ao tema e subtemas.
            Ao coletar os dados, documentos e links, separe em blocos esses conteúdos, cada bloco com um tema, e dentro de cada bloco, coloque os dados (textos, documentos e links) coletados organizando os dados nos blocos que os dados correspondam ao tema do respectivo bloco.
        """
    )

    entrada_do_ctb_coletor = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(coletor_de_dados, entrada_do_ctb_coletor)
    return plano_do_post

In [55]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def ctb_3_redator(topico, coletar_dados):
    redator_de_dados = Agent(
        name="ctb_3_redator",
        model="gemini-2.0-flash",
        description="Agente responsável por organisar todo conteúdo gerado, e aplicar as regras de identação do documento",
        instruction="""
          Regras:
            1. Seja:
              Minusioso.
              Cuidadoso.
              Analista.
            2. Faça revisões ao produzir o contúdo solicitado.
            3. Não use palavras ou termos alegóricos.
            4. Seja: pragmático; sério; objetivo; questionador; imparcial; realista.
            5. Ao entregar um resultado por texto, não faça uma abordagem muito longa, vá direto ao assunto principal, seja perspicaz e pragmático em o faze-lo
            6. Ao usar termos técnicos, colocando o significado do termo entre parenteses. Exemplo: Este rapáz é obtuso "(pouco inteligente)" em dizer tamanha loucura.
            7. O conteúdo deve ser organizado, tendo 1 tema principal com uma breve descrição de até 240 caracteres. Após a descrição do tema principal, deve seguir com os subtemas e seus respectivos tópicos internos. além disso, cada tópico deve tex um exemplo explicativo para ajudar o aluno a se contestualizar melhor com o conteúdo.

          Tarefas:
            tarefa_1:
              Analise od dados gerados pelo "ctb_2_dados".
              Verifique se está havendo: Redundancia; Repetições, desnecessárias; conteúdo fora do seu contexto.
              Organise os dados em uma sequencia lógica, que seja fiél ao tema principal. Use a regra número "7" como base principal para realisar essa tarefa.

            """
    )
    entrada_do_agente_redator_de_dados = f"Tópico: {topico}\nOrganizar dados: {coletar_dados}"
    # Executa o agente
    rascunho = call_agent(redator_de_dados, entrada_do_agente_redator_de_dados)
    return rascunho

In [64]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def ctb_4_explicador(topico, organizar_dados):
    ctb_explicador = Agent(
        name="ctb_4_explicador",
        model="gemini-2.0-flash",
        description="Agente responsávl por explicar para o aluno o conteúdo.",
        instruction="""
            Regras:
              1. Divida o conteúdo em: Tema principal - subtemas - tópicos e exemplos. A seguir um exemplo para uma melhor compreensão.
              6. Seja objetivo e centrado no conteúdo que está sendo abordado.

            tarefa_1: Após ter feito a divisões, divicão essa que foi feita na regra número "1".
            """
    )
    entrada_do_agente_explicador_do_assunto = f"Tópico: {topico}\nRascunho: {organizar_dados}"
    # Executa o agente
    texto_revisado = call_agent(ctb_explicador, entrada_do_agente_explicador_do_assunto)
    return texto_revisado

In [65]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Vamos aprender! 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("Oque voce quer aprender?: ")

# Inserir lógica do sistema de agentes ######
if not topico:
  print("Voce não disse oque quer estudar. Pode fazer uma descrição simples, que eu faço o resto.")

else:
    print(f"Hum! Ótimo tema, vamos lá então {topico}")



#####    ---------------------  x X x  -------------------------    ######



    lancamentos = ctb_1_prompt(topico, data_de_hoje)


    print("\n--- Resultado do Agente 1 (ctb_1_prompt) ---\n")
    display(to_markdown(lancamentos))



#####    ---------------------  x X x  -------------------------    ######



    coletar_dados = ctb_2_dados(topico, lancamentos)


    print("\n--- Resultado do Agente 2 (ctb_2_dados) ---\n")
    display(to_markdown(coletar_dados))



#####    ---------------------  x X x  -------------------------    ######



    organizar_dados = ctb_3_redator(topico, coletar_dados)


    print("\n--- Resultado do Agente 2 (ctb_3_redator) ---\n")
    display(to_markdown(organizar_dados))



#####    ---------------------  x X x  -------------------------    ######



    professor = ctb_4_explicador(topico, organizar_dados)



    print("\n--- Resultado do Agente 2 (ctb_3_redator) ---\n")
    display(to_markdown(professor))

    print("----------------------------------")


Vamos aprender! 🚀
Oque voce quer aprender?: me explique oque é um átomo


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-177' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pyth

Hum! Ótimo tema, vamos lá então me explique oque é um átomo

--- Resultado do Agente 1 (ctb_1_prompt) ---



> Okay, vamos processar sua solicitação.
> 
> **tarefa_1:**
> 
> *   **Prompt:** "Tópico: me explique oque é um átomo"
> *   **Tipo de Comando:** Pergunta
> 
> Informação_1 Salva.
> 
> **tarefa_2:**
> 
> Analisando a Informação_1:
> 
> *   **Tema Central:** O átomo (sua definição e conceito básico).
> *   **Formato de Resposta Esperado:** Texto explicativo (abordagem didática, provavelmente buscando uma explicação clara e acessível).
> *   **Objetivo Didático:** Entender a definição e o conceito fundamental de um átomo. O aluno busca uma explicação introdutória sobre o que é um átomo.
> 
> **tarefa_3:**
> 
> Novo Prompt:
> 
> "Explique o que é um átomo de forma clara e didática. Inclua os componentes básicos de um átomo (prótons, nêutrons e elétrons) e onde eles se localizam. Use uma linguagem simples, como se estivesse explicando para alguém que nunca ouviu falar sobre o assunto antes."
> 



--- Resultado do Agente 2 (ctb_2_dados) ---



...


--- Resultado do Agente 2 (ctb_3_redator) ---



> ## Átomo: A Unidade Fundamental da Matéria
> 
> O átomo é a menor unidade de um elemento químico que mantém suas propriedades. Essencialmente indivisível na visão original, hoje sabemos que é composto por partículas ainda menores.
> 
> ### Componentes Atômicos
> 
> 1.  **Prótons:**
>     *   **Localização:** Núcleo atômico (região central).
>     *   **Carga:** Positiva (+).
>     *   **Função:** Determina o elemento químico. O número de prótons é único para cada elemento.
> 
>         *   **Exemplo:** Todo átomo com 1 próton é hidrogênio; com 6 prótons, é carbono.
> 2.  **Nêutrons:**
>     *   **Localização:** Núcleo atômico (junto com os prótons).
>     *   **Carga:** Neutra (sem carga elétrica).
>     *   **Função:** Contribui para a estabilidade do núcleo. Variações no número de nêutrons criam isótopos.
> 
>         *   **Exemplo:** Carbono-12 e Carbono-14 são isótopos do carbono, com diferentes números de nêutrons.
> 3.  **Elétrons:**
>     *   **Localização:** Eletrosfera (região ao redor do núcleo).
>     *   **Carga:** Negativa (-).
>     *   **Função:** Define as propriedades químicas e interações com outros átomos.
> 
>         *   **Exemplo:** A forma como os elétrons se organizam permite que o átomo de hidrogênio se ligue ao oxigênio para formar água.
> 
> ### Estrutura Atômica
> 
> 1.  **Núcleo:**
>     *   Concentra quase toda a massa do átomo.
>     *   Composto por prótons e nêutrons.
>     *   Extremamente pequeno e denso.
> 2.  **Eletrosfera:**
>     *   Região onde os elétrons orbitam o núcleo.
>     *   Predominantemente espaço vazio.
>     *   Elétrons ocupam orbitais (regiões de probabilidade), não caminhos fixos.
> 
>         *   **Exemplo:** A eletrosfera do átomo de oxigênio contém orbitais que acomodam seus elétrons, permitindo que ele se ligue a outros átomos.
> 



--- Resultado do Agente 2 (ctb_3_redator) ---



> ## Átomo: A Unidade Fundamental da Matéria
> 
> *   **Tema Principal:** Átomo - A menor unidade de um elemento que mantém suas propriedades.
> 
>     *   **Subtemas:**
> 
>         1.  Componentes Atômicos
>         2.  Estrutura Atômica
>     *   **Tópicos e Exemplos:**
> 
>         1.  **Componentes Atômicos:**
> 
>             *   **Prótons:**
> 
>                 *   **Localização:** Núcleo atômico.
>                 *   **Carga:** Positiva (+).
>                 *   **Função:** Determina o elemento químico.
>                 *   **Exemplo:** Átomos com 1 próton são hidrogênio, com 6 prótons são carbono.
>             *   **Nêutrons:**
> 
>                 *   **Localização:** Núcleo atômico.
>                 *   **Carga:** Neutra (sem carga elétrica).
>                 *   **Função:** Contribui para a estabilidade do núcleo.
>                 *   **Exemplo:** Carbono-12 e Carbono-14 (isótopos com diferentes números de nêutrons).
>             *   **Elétrons:**
> 
>                 *   **Localização:** Eletrosfera.
>                 *   **Carga:** Negativa (-).
>                 *   **Função:** Define as propriedades químicas e interações com outros átomos.
>                 *   **Exemplo:** Elétrons permitem que o hidrogênio se ligue ao oxigênio para formar água.
>         2.  **Estrutura Atômica:**
> 
>             *   **Núcleo:**
> 
>                 *   Concentra quase toda a massa do átomo.
>                 *   Composto por prótons e nêutrons.
>                 *   Extremamente pequeno e denso.
>             *   **Eletrosfera:**
> 
>                 *   Região onde os elétrons orbitam o núcleo.
>                 *   Predominantemente espaço vazio.
>                 *   Elétrons ocupam orbitais (regiões de probabilidade).
>                 *   **Exemplo:** A eletrosfera do oxigênio contém orbitais que acomodam seus elétrons, permitindo ligações com outros átomos.
> 


----------------------------------
